#### Use LLM to output the contextual information present in a text

Requirement:
* Interact with OpenAI model using API
* Use LLM for understanding the context in a text
* Use LLM to solve one possible use case

In [148]:
# Interact with OpenAI API
import os
import openai

# Securly save the API key locally
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
# Write a helper function to receive the response
llm_model = "gpt-3.5-turbo"
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # for minimal randomness
    )
    return response.choices[0].message["content"]

In [150]:
# Sample response
get_completion("Are you available to assist?")

'Yes, I am here to assist you. How can I help you today?'

#### Use LLM to understand the context

In [151]:
# LLMs understand the context well - which is one of very useful application of it.
# In this below example, we will try to demonstrate
# The below email content was taken from huggingface/enron - dataset as an example

text_content = """
Traveling to have a business meeting takes the fun out of the trip.\
Especially if you have to prepare a presentation. I would suggest holding \
the business plan meetings here then take a trip without any formal business meetings.\
I would even try and get some honest opinions on whether a trip is even desired or \
necessary. As far as the business meetings, I think it would be more productive to try\
and stimulate discussions across the different groups about what is working and what is not.\
Too often the presenter speaks and the others are quiet just waiting for their turn. \
The meetings might be better if held in a round table discussion format.\
My suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. \
Flying somewhere takes too much time. John Mcthy is involved in fraud investigation\
"""

question = """Is this text content has any indication of any crime or confiction?. \
    If yes, respond with 'yes'. If no, respond with 'no'\
    If yes who has committed?, if no, just leave empty space
    If yes, is it a person or organization involved in it?.
    Simply provide these responses separeted by comma"""

prompt = f"""Answer the question that is : {question} \
    using the text that is delimited by triple backticks  \
    text: ```{text_content}```
"""
print(prompt)

Answer the question that is : Is this text content has any indication of any crime or confiction?.     If yes, respond with 'yes'. If no, respond with 'no'    If yes who has committed?, if no, just leave empty space
    If yes, is it a person or organization involved in it?.
    Simply provide these responses separeted by comma     using the text that is delimited by triple backticks      text: ```
Traveling to have a business meeting takes the fun out of the trip.Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings.I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to tryand stimulate discussions across the different groups about what is working and what is not.Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be bet

In [152]:
response = get_completion(prompt)
response

'yes, John Mcthy, person'

#### Interact with model

In [153]:
# Use langchain library for the use case
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [154]:
# Generate the prompt as a string with inputs closed with {}
prompt = """Answer the question that is : {question} using the text\
      that is delimited by triple backticks  text: ```{text_content}```
"""
prompt_template = ChatPromptTemplate.from_template(prompt)

In [178]:
# Template takes two parameters as input
prompt_template.messages[0].prompt.input_variables

['question', 'text_content']

In [157]:
# Prompt template
news_text_context = prompt_template.format_messages(
                    question=question,
                    text_content=text_content)

In [179]:
# Call the LLM to translate to the style of the customer message
news_text_context_output = chat(news_text_context)
print(news_text_context_output.content)

```
yes, John Mcthy, person
```


#### Extracting the outputs in needed format

In [165]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [166]:
result_schema = ResponseSchema(name="result",
                             description="Is this text content has any \
                                indication of any crime or confiction?. \
                                If yes, respond with 'yes'. If no, respond \
                                with 'no'")
name_schema = ResponseSchema(name="name",
                              description="If yes who has committed?,\
                              if no, just leave empty space")
entity_type_schema = ResponseSchema(name="entity",
                                    description="If yes, is it a person or organization\
                                    involved in it?")

response_schemas = [result_schema, 
                    name_schema,
                    entity_type_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [169]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"result": string  // Is this text content has any                                 indication of any crime or confiction?.                                 If yes, respond with 'yes'. If no, respond                                 with 'no'
	"name": string  // If yes who has committed?,                              if no, just leave empty space
	"entity": string  // If yes, is it a person or organization                                    involved in it?
}
```


In [180]:
template_alternate = """\
For the following text, extract the following information:

result: Is this text content has any \
    indication of any crime or confiction?. \
    If yes, respond with 'yes'. If no, respond \
    with 'no'

name: If yes who has committed?,\
    if no, just leave empty space.

entity: If yes, is it a person or organization\
        involved in it?

text: {text_content}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template_alternate)

messages = prompt.format_messages(text_content=text_content, 
                                format_instructions=format_instructions)

In [181]:
print(messages[0].content)

For the following text, extract the following information:

result: Is this text content has any     indication of any crime or confiction?.     If yes, respond with 'yes'. If no, respond     with 'no'

name: If yes who has committed?,    if no, just leave empty space.

entity: If yes, is it a person or organization        involved in it?

text: 
Traveling to have a business meeting takes the fun out of the trip.Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings.I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to tryand stimulate discussions across the different groups about what is working and what is not.Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format.My sug

In [182]:
response = chat(messages)
print(response.content)

```json
{
	"result": "yes",
	"name": "John Mcthy",
	"entity": "person"
}
```


In [183]:
output_dict = output_parser.parse(response.content)
type(output_dict)

dict

In [ ]:
# Check the output for correctness
output_dict

{'result': 'yes', 'name': 'John Mcthy', 'entity': 'person'}

In [ ]:

output_dict['name']

'John Mcthy'